**Project group 13: feature extraction for audio**  
CMPT 419/983  
Summer 2020  

*Credit: This notebook was modified starting from weekly activity 6's notebook. The weekly activity 6 notebook was prepared by TA Payam Jome Yazdian*

In [ ]:
PROJECT_FOLDER = "/content/drive/My\ Drive/school-419-project/"

# Import dependencies

The speech recognition and textgrid dependencies are commented out.

In [ ]:
!pip3 install soundfile 
!pip3 install praat-parselmouth
#!pip3 install acoustics
#!pip3 install tgt
#!pip3 install praat-textgrids
#!pip3 install pydub
#!pip3 install SpeechRecognition

import math
import numpy as np
import soundfile as sf
import matplotlib as mpl
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd
from IPython.display import Audio
import pandas as pd

import os
import sys
import parselmouth
#from acoustics.cepstrum import real_cepstrum
#import tgt
#from pydub import AudioSegment
#from pydub.silence import split_on_silence
#import speech_recognition as sr
#import textgrids


Mount our drive. Copied from Dylan's code in https://colab.research.google.com/drive/1_YtiBCoB4-HEQm0M2rNqgAfYXdZfB3FK#scrollTo=JqmmTyLGFXRh&line=2&uniqifier=1

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd {PROJECT_FOLDER}

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/school-419-project


Print the working directory and list its files.

In [ ]:
!pwd
!ls -l

/content/drive/My Drive/school-419-project
total 62974
-rw------- 1 root root 61589481 Aug 16 05:57 aggregated_openface_data.pickle
-rw------- 1 root root  1907690 Aug 15 06:28 audio-features.csv
-rw------- 1 root root   974877 Aug 15 06:28 audio-features.pickle
drwx------ 2 root root     4096 Aug 16 04:48 audio_output
drwx------ 2 root root     4096 Aug 16 04:35 clipped_videos
drwx------ 4 root root     4096 Aug 15 04:14 openface_processed


Convert the videos into .wav files. Copied from Dylan's code in https://colab.research.google.com/drive/1_YtiBCoB4-HEQm0M2rNqgAfYXdZfB3FK#scrollTo=b_KDtLp0Oy4Z&line=3&uniqifier=1

Escape the spaces because ffmpeg needs escaped paths.

In [ ]:
def extract_basename(filename):
  basename, _ = os.path.splitext(filename)
  return basename

def escape_spaces(filename):
  return filename.replace(" ", "\\ ")

def unescape_spaces(filename):
  return filename.replace("\\ ", " ")

VIDEO_SOURCE_FOLDER = PROJECT_FOLDER + "clipped_videos/"
AUDIO_OUTPUT_FOLDER = PROJECT_FOLDER + "audio_output/"
TEXT_OUTPUT_FOLDER = PROJECT_FOLDER + "text_output/"
VIDEO_SOURCE_FOLDER_UNESCAPED = unescape_spaces(VIDEO_SOURCE_FOLDER)
AUDIO_OUTPUT_FOLDER_UNESCAPED = unescape_spaces(AUDIO_OUTPUT_FOLDER)
TEXT_OUTPUT_FOLDER_UNESCAPED = unescape_spaces(TEXT_OUTPUT_FOLDER)

In [ ]:
if not os.path.exists(AUDIO_OUTPUT_FOLDER_UNESCAPED):
  os.makedirs(AUDIO_OUTPUT_FOLDER_UNESCAPED)

if not os.path.exists(TEXT_OUTPUT_FOLDER_UNESCAPED):
  os.makedirs(TEXT_OUTPUT_FOLDER_UNESCAPED)

for root, _, files in os.walk(VIDEO_SOURCE_FOLDER_UNESCAPED):
  root_escaped = escape_spaces(root)
  for f in files:
    source = root_escaped + f
    basename = extract_basename(f) + '.wav'
    output = AUDIO_OUTPUT_FOLDER + basename

    # Use -n to never overwrite the output file if it exists
    !ffmpeg -n -i {source} {output}

Speech recognition/Pydub has issues with HD audio, so reduce the number of channels to 2 if the audio file has > 2 channels (https://github.com/jiaaro/pydub/issues/129). We reduce the 6 channel HD audio to 2 channels (https://trac.ffmpeg.org/wiki/AudioChannelManipulation). Pass the Python variable AUDIO_OUTPUT_FOLDER_UNESCAPED to bash as a string, as $1. The code is commented because we are not doing speech recognition.

In [ ]:
# %%bash -s "$AUDIO_OUTPUT_FOLDER_UNESCAPED"
# AUDIO_OUTPUT_FOLDER_UNESCAPED=$1

# # If the variable's value contains spaces, we must surround the variable with quotes
# for file in "$AUDIO_OUTPUT_FOLDER_UNESCAPED"*;
# do
#   CHANNELS=$(ffprobe -i "$file" -show_entries stream=channels -select_streams a:0 -of compact=p=0:nk=1 -v 0)
#   if [[ $CHANNELS -gt 2 ]];
#   then
#     echo "Converting ${file} from ${CHANNELS} channels to 2 channels"
#     # Overwrite the file if it already exists with -y
#     ffmpeg -i "$file" -ac 2 "$file" -y
#     # Also double the volume since the audio is quiet
#     ffmpeg -i "$file" -filter:a "volume=2" "$file" -y
#   else
#     echo "Did not convert ${file}"
#   fi
# done

# Visualize a wave signal

These spectogram and cepstrum plots only display non-negative frequencies and times because the plots are symmetric around the y-axis.

In [ ]:
# plot spectogram:
def plot_spectogram(data, sampling_rate, filename, title_extra=''):
    spectogram = librosa.feature.melspectrogram(y=data, sr=sampling_rate)
    spectogram_dB = librosa.power_to_db(spectogram, ref=np.max)
    #plt.figure(figsize=(10, 4))
    librosa.display.specshow(spectogram_dB,
                             x_axis='time',
                             y_axis='mel',
                             sr=sampling_rate,
                             fmax=8000)
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel-frequency spectrogram' + title_extra)
    plt.tight_layout()
    plt.show()

In [ ]:
# plot cepstrum:
def plot_cepstrum(data, sampling_rate, title_extra=''):
    frame_size = len(data)
    positive_axis = range(0, frame_size // 2)
    dt = 1.0 / sampling_rate
    freq_vector = np.fft.fftfreq(frame_size, d=dt)

    freq = np.fft.fft(data)
    amp = np.abs(freq)
    log_amp = np.log(amp)

    fig, ax = plt.subplots()
    ax.plot(freq_vector[positive_axis], amp[positive_axis])
    ax.set_xlabel('frequency (Hz)')
    ax.set_title('Fourier spectrum')
    plt.show()

    # Take the inverse DFT of the log_amp
    cepstrum = np.fft.ifft(log_amp)
    cepstrum_amp = np.abs(cepstrum)
    df = freq_vector[1] - freq_vector[0]
    quefrency_vector = np.fft.fftfreq(log_amp.size, df)

    # Plot the cepstrum
    fig, ax = plt.subplots()
    ax.plot(quefrency_vector[positive_axis], cepstrum_amp[positive_axis])
    ax.set_xlabel('quefrency (s)')
    ax.set_title('cepstrum calculated with numpy')

    # Zoom in on the y-axis
    ax.set_ylim([0, 0.5])
    plt.show()

# Plot the spectrum and cepstrum

Each cepstrum's y axis range was set to [0, 0.5] to zoom in on the smaller bumps.

In [ ]:
for root, _, files in os.walk(AUDIO_OUTPUT_FOLDER_UNESCAPED):
  for f in files:
    output = root + f

    file_info = sf.info(output)
    print(file_info, end='') # print audio file information
    print(":")

    # Play the audio.
    ipd.display(ipd.Audio(output))

    data, sample_rate = librosa.load(output, sr=None)
    librosa.display.waveplot(data) # Plot audio file in wave format

    # Plot spectrum and cepstrum of the wave file.
    plot_spectogram(data, sample_rate, f, '')
    plot_cepstrum(data, sample_rate, '')

    print("==================================")

# Feature extraction

## Mean, minimum, and maximum pitch

In [ ]:
def get_pitch(sound):
  pitch = sound.to_pitch(pitch_floor=180.0)

  pitch_values = pitch.selected_array['frequency']
  pitch_values[pitch_values == 0] = np.nan
  # Take the mean while ignoring NaNs
  mean_pitch = np.nanmean(pitch_values)

  min_pitch = parselmouth.praat.call(pitch, "Get minimum", 0, 0, "Hertz", "None")
  max_pitch = parselmouth.praat.call(pitch, "Get maximum", 0, 0, "Hertz", "None")

  return (mean_pitch, min_pitch, max_pitch)

def print_pitch(description, value, rounding=5):
  print(f"The audio file's {description} pitch is {value:.{rounding}f} Hz")

## Exploring formants in speech


### Speech recognition (unused)

Use Google's speech recognition API to return the most likely audio transcription. If the API couldn't recognize the text, create an empty text file. Place each word on a newline in the output text file so that forced alignment will output separate time periods for each word.

In [ ]:
# # initialize the recognizer
# r = sr.Recognizer()

# def recognize_text(audio_filename, text_output_filename, recognizer=r):
#   with sr.AudioFile(audio_filename) as source:
#       # listen for the data (load audio to memory)
#       audio_data = r.record(source)
#       # recognize (convert from speech to text)
#       try:
#         text = r.recognize_google(audio_data)
#       except sr.UnknownValueError:
#         print("Speech unintelligible.")
#         text = ""
#       except sr.RequestError:
#         print("The speech recognition operation failed, the key isn't valid, or there is no internet connection.")
#         text = ""

#       output = '\n'.join(text.split())
#       print(output + '\n')

#       with open(text_output_filename, 'w') as text_file:
#         text_file.write(output)
#   return text
#
# for root, _, files in os.walk(AUDIO_OUTPUT_FOLDER_UNESCAPED):
#   for f in files:
#     basename = extract_basename(f)
#     video_filename = VIDEO_SOURCE_FOLDER + basename + ".mp4"
#     audio_filename = root + f
#     text_output_filename = TEXT_OUTPUT_FOLDER + basename + ".txt"
#     text_output_filename_unescaped = unescape_spaces(text_output_filename)

#     print(audio_filename)
#     ipd.display(ipd.Audio(audio_filename))
#     text = recognize_text(audio_filename, text_output_filename_unescaped)
#     print("Speech to text:", text)
#     print("================================================")

Print all the text the Google speech recognition API recognized from the speech. The accuracy is decent if there was not much background music and if the speech was clear. 12/20 of the audio files had intelligible speech. It is time-consuming to correct the recognized text and transcribe the audio, so we will not do speech recognition.

In [ ]:
# %%bash -s "$TEXT_OUTPUT_FOLDER_UNESCAPED"
# TEXT_OUTPUT_FOLDER_UNESCAPED=$1

# # If the variable's value contains spaces, we must surround the variable with quotes
# for file in "$TEXT_OUTPUT_FOLDER_UNESCAPED"*;
# do
#   echo "$file:"
#   cat "$file"
#   echo -e '\n'
# done

### Forced alignment of words (unused)

Install the aeneas Python library for forced alignment. Forced alignment is mapping words, sentence fragments, or sentences to their timespan in the audio file.

In [ ]:
# !apt-get install -y espeak espeak-data libespeak1 libespeak-dev
# !pip3 install aeneas
# !python -m aeneas.diagnostics

Calculate the forced alignment. Use a small MFCC window shift to achieve word-level granularity (https://github.com/readbeyond/aeneas/blob/master/wiki/HOWITWORKS.md).

In [ ]:
# !python -m aeneas.tools.execute_task \
#     /content/drive/My\ Drive/school-419-project/audio_output/flirt-qUxXbU2ModY-0.wav \
#     /content/drive/My\ Drive/school-419-project/text_output/flirt-qUxXbU2ModY-0.txt \
#     "task_language=eng|os_task_file_format=textgrid|is_text_type=plain|mfcc_window_length=0.150|mfcc_window_shift=0.050" \
#     map.textgrid

# # Try to open the file as textgrid
# grid = textgrids.TextGrid(unescape_spaces(PROJECT_FOLDER) + "map.textgrid")

# # Assume "Token" is the name of the tier
# # containing word information
# intervalTier = grid['Token']
# for interval in intervalTier:
#   # Convert Praat notation to Unicode. If the input is not in Praat notation,
#   # nothing should happen.
#   label = interval.text.transcode()
#   # Print label and syllable duration, CSV-like
#   print('"{}";{}'.format(label, interval.dur))

Print the forced alignment.

In [ ]:
# %cat {PROJECT_FOLDER}/map.textgrid

### Formants

I am basing the below code on https://homepage.univie.ac.at/christian.herbst/python/#formantDemo. Instead of using the library linked, Parselmouth is used. The formant plot has a point for each frame.

In [ ]:
def plot_f1_f2(f1_frequencies, f2_frequencies, basename):
  plt.scatter(f1_frequencies, f2_frequencies, alpha=0.4)
  plt.xlabel("F1 [Hz]")
  plt.ylabel("F2 [Hz]")
  plt.title("F1/F2 plot for " + basename)

  # Hardcode the maximum frequencies so that each plot has the same axis
  plt.xlim(0, 3000)
  plt.ylim(0, 4000)
  plt.show()

def get_mean_frequencies_of_f1_and_f2(sound, basename):
  formant = sound.to_formant_burg()
  n = formant.get_number_of_frames()

  # Get the frequencies of formant 1 and 2 at each frame
  f1_frequencies = []
  f2_frequencies = []

  # Add 1 to the range's start and end indices because the frame number must be
  # a positive integer
  for i in range(1, n + 1):
    time = formant.frame_number_to_time(i)

    # Extract the frequency of formant 1 and formant 2
    f1 = formant.get_value_at_time(1, time)
    f2 = formant.get_value_at_time(2, time)
    f1_frequencies.append(f1)
    f2_frequencies.append(f2)
  
  #plot_f1_f2(f1_frequencies, f2_frequencies, basename)

  mean_f1_frequency = np.mean(f1_frequencies)
  mean_f2_frequency = np.mean(f2_frequencies)
  return mean_f1_frequency, mean_f2_frequency

## Mean speech rate

Slower speech has more pauses than rapid speech ([The Duration of Speech Pauses in a Multilingual Environment by Demol, Verhelst, and Verhoeve](https://dblp.uni-trier.de/rec/bibtex/conf/interspeech/DemolVV07)), so 1 / (# pauses) is proportional to the speech rate. If the sound has 1 pause because Praat could not detect any speech or if there was actually one pause, then the ratio is 1, so divide by the sound's duration. We are using Praat's default parameters for the command To TextGrid (silences).

In [ ]:
MINIMUM_PITCH = 100
TIME_STEP = 0
SILENCE_THRESHOLD = -25.0
MINIMUM_SILENT_INTERVAL_DURATION = 0.1
MINIMUM_SOUNDING_INTERVAL_DURATION = 0.1
SILENT_INTERVAL_LABEL = "silent"
SOUNDING_INTERVAL_LABEL = "sounding"

# There is only the silences tier
TIER_NUMBER = 1

def get_mean_speech_rate(sound):
  data = parselmouth.praat.call(sound, "To TextGrid (silences)...",
                                MINIMUM_PITCH,
                                TIME_STEP,
                                SILENCE_THRESHOLD,
                                MINIMUM_SILENT_INTERVAL_DURATION,
                                MINIMUM_SOUNDING_INTERVAL_DURATION,
                                SILENT_INTERVAL_LABEL,
                                SOUNDING_INTERVAL_LABEL)

  # Get the number of intervals for the silences tier
  number_of_silent_intervals = parselmouth.praat.call(data, "Get number of intervals...", TIER_NUMBER)
  duration = sound.get_total_duration()

  mean_speech_rate = 1 / (number_of_silent_intervals * duration)

  # Praat couldn't find any speech
  #if number_of_silent_intervals == 1:
  #  mean_speech_rate = None
  return mean_speech_rate, number_of_silent_intervals

# Try to get features per video frame

Put the audio features in a dataframe.

The get_intensity() method gets the sound's mean intensity and is in units of decibels (dB). It calls Sound_getIntensity_dB in https://github.com/YannickJadoul/Parselmouth/blob/3b4ae221618bc7a5c47f5d7e08e118680a8ca549/praat/fon/Sound.cpp#L311.

Harmonicity represents how much energy a sound has in its periodic part versus its noise part and is also called the Harmonics-to-Noise Ratio (HNR) -- source https://www.fon.hum.uva.nl/praat/manual/Harmonicity.html.

A spectrum is the frequency-domain version of the sound, transformed using the Fourier transform (https://www.fon.hum.uva.nl/praat/manual/Sound__To_Spectrum___.html). Its center of gravity represents how high the frequencies are (source: https://www.fon.hum.uva.nl/praat/manual/Sound__To_Spectrum___.html). The power is 2 for all the spectrum features (source: Parselmouth documentation).

In [ ]:
#num_audio_frames/num_video_frames

809.7078651685393

In [ ]:
DF_COLUMNS = [
  "mean_pitch",
  "min_pitch",
  "max_pitch",
  "mean_intensity",
  "root_mean_square",
  "center_of_gravity",
  "kurtosis",
  "skewness",
  "standard_deviation",
  "mean_f1_frequency",
  "mean_f2_frequency",
  "mean_speech_rate",
  "number_of_silent_intervals",
  "filename",
  "estimated_frame_number",
]

# Use a list of dicts to create the dataframe because it's the fastest way to
# create a dataframe row by row
df_row_list = []
for root, _, files in os.walk(AUDIO_OUTPUT_FOLDER_UNESCAPED):
  for f in files:
    basename = extract_basename(f)
    video_filename = VIDEO_SOURCE_FOLDER + basename + ".mp4"
    audio_filename = root + f

    sound = parselmouth.Sound(audio_filename)
    num_audio_frames = sound.get_number_of_frames()

    video_csv = pd.read_csv(f"openface_processed/multi/{basename}.csv")
    num_video_frames = np.shape(video_csv)[0]

    print(f"Basename: {basename}\nAudio: {num_audio_frames}\nVideo: {num_video_frames}\n")

    times = []
    for frame_segment in range(1, num_audio_frames+1, num_audio_frames//num_video_frames):
      #print(f"Trying segment {frame_segment}")
      times.append(sound.frame_number_to_time(frame_segment))

    timepoints = []
    for i in range(len(times)-30):
      timepoints.append((times[i], times[i+30]))


    print(f"Avg sample length: {(times[1] - times[0])*1000}ms")

    print(timepoints)

    soundparts = []
    # split into audio sections per frame
    for i, (fromtime, totime) in enumerate(timepoints):
      soundparts.append(sound.extract_part(from_time=fromtime, to_time=totime))

    for estimated_frame_number, sound in enumerate(soundparts):
      mean_pitch, min_pitch, max_pitch = get_pitch(sound)
      mean_intensity = sound.get_intensity()

      #harmonicity = sound.to_harmonicity()
      # Source: https://parselmouth.readthedocs.io/en/stable/examples/batch_processing.html
      #mean_harmonicity = harmonicity.values[harmonicity.values != -200].mean()
      #mean_harmonicity = 0

      root_mean_square = sound.get_root_mean_square()

      spectrum = sound.to_spectrum()
      center_of_gravity = spectrum.get_centre_of_gravity()
      kurtosis = spectrum.get_kurtosis()
      skewness = spectrum.get_skewness()
      standard_deviation = spectrum.get_standard_deviation()

      # Plot an F1/F2 graph for each audio file
      mean_f1_frequency, mean_f2_frequency = get_mean_frequencies_of_f1_and_f2(sound, basename)

      mean_speech_rate, number_of_silent_intervals = get_mean_speech_rate(sound)
      
      row_dict = {DF_COLUMNS[0]: mean_pitch,
                  DF_COLUMNS[1]: min_pitch,
                  DF_COLUMNS[2]: max_pitch,
                  DF_COLUMNS[3]: mean_intensity,
                  DF_COLUMNS[4]: root_mean_square,
                  DF_COLUMNS[5]: center_of_gravity,
                  DF_COLUMNS[6]: kurtosis,
                  DF_COLUMNS[7]: skewness,
                  DF_COLUMNS[8]: standard_deviation,
                  DF_COLUMNS[9]: mean_f1_frequency,
                  DF_COLUMNS[10]: mean_f2_frequency,
                  DF_COLUMNS[11]: mean_speech_rate,
                  DF_COLUMNS[12]: number_of_silent_intervals,
                  DF_COLUMNS[13]: basename,
                  DF_COLUMNS[14]: estimated_frame_number}
      df_row_list.append(row_dict)

  df = pd.DataFrame(df_row_list, columns=DF_COLUMNS)
  df

Write the dataframe to a CSV file and to a pickle file.

In [ ]:
df.to_csv(unescape_spaces(PROJECT_FOLDER) + "audio-features.csv")
df.to_pickle(unescape_spaces(PROJECT_FOLDER) + "audio-features.pickle")

Websites referred to

https://flothesof.github.io/cepstrum-pitch-tracking.html  
https://numpy.org/doc/stable/reference/routines.fft.html  
https://python-acoustics.github.io/python-acoustics/cepstrum.html  
https://billdthompson.github.io/assets/output/Jadoul2018.pdf  
https://readthedocs.org/projects/parselmouth/downloads/pdf/latest/